# Performance comparisons

In memory and out of memory, using dask.

In [23]:
# Change dir to repo root if running from repo (rather than pip installed)
# (Assuming running from [repo]/scripts/)
import os
os.chdir('../')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import matplotlib.pyplot as plt
import numpy as np

from typing import Tuple

from incremental_trees.trees import StreamingRFC

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import dask_ml.datasets
from dask_ml.wrappers import Incremental
from dask.distributed import Client, LocalCluster
from dask_ml.model_selection import train_test_split as dask_tts

import dask as dd
import pandas as pd

In [25]:
# Settings
MAX_ESTIMATORS = 60  # Lower to run faster

In [26]:
# Prepare dask cluster
cluster = LocalCluster(
    processes=False,
    n_workers=2,
    threads_per_worker=2,
    scheduler_port=8383
)
client = Client(cluster)
client

2023-01-01 21:04:41,365 - distributed.deploy.spec - WARNING - Cluster closed without starting up


RuntimeError: Cluster failed to start: Scheduler failed to start.

# Synthetic data, in memory

Compare increasing estimators with RandomForest (using warm_start) against Incremental StreamingRFC (dask handles .partial_fit).

In [ ]:
x, y = dask_ml.datasets.make_blobs(
    n_samples=1e5,
    chunks=1e4,
    random_state=0,
    n_features=40,
    centers=2,
    cluster_std=100
)

x_dd = dd.dataframe.from_array(x, chunksize=1e4)
y_dd = dd.dataframe.from_array(y, chunksize=1e4)

x_pd = pd.DataFrame(x.persist().compute())
y_pd = pd.DataFrame(y.persist().compute())

In [ ]:
x_pd.memory_usage(deep=True).sum() / 1024 / 1024

### Standard random forest

In [ ]:
def score(mod, train: Tuple[np.array, np.array], test: Tuple[np.array, np.array], pr=False) -> Tuple[float, float]:
    """Return ROC auc on x_train and x_test (from caller) on mod. Print if requested."""

    y_pred_train_proba = mod.predict_proba(train[0])[:, 1]
    y_pred_test_proba = mod.predict_proba(test[0])[:, 1]

    roc_train = roc_auc_score(train[1], y_pred_train_proba)
    roc_test = roc_auc_score(test[1], y_pred_test_proba)
    if pr:
        print(f"n_ests: {len(rfc.estimators_)}")
        print(f'Train AUC: {roc_train}')
        print(f'Test AUC: {roc_test}')

    return roc_train, roc_test


def score_dask(mod, train: Tuple[np.array, np.array], test: Tuple[np.array, np.array], pr=False) -> Tuple[float, float]:
    """Score model using available dask metric (accuracy)."""

    roc_train = mod.score(train[0], train[1])
    roc_test = mod.score(test[0], test[1])
    if pr:
        print(f"n_ests: {len(rfc.estimators_)}")
        print(f'Train AUC: {roc_train}')
        print(f'Test AUC: {roc_test}')

    return roc_train, roc_test


def multiple_fit(x: np.array, y: np.array, steps=np.arange(1, 101, 2), sample: int = 1):
    """
    Fit a random forest model with an increasing number of estimators.
    
    This version doesn't use warm start and refits the model from scratch each iteration.
    This is for the sake of comparing timings to dask function below.
    
    :param steps: Range to iterate over. Sets total number of estimators that will be fit in model
                  after each iteration. Should be range with constant step size.
    :param sample: Proportion of randomly sampled training data to use on each partial_fit call.
                   If sample = 1, all training data is used on each interation,
                   so should behave as standard random forest. Default = 1 (100%).
    """

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1)

    train_scores = []
    test_scores = []
    for s in steps:
        # Fit full model on each iteration
        rfc = RandomForestClassifier(warm_start=False)

        # Fit model with these n ests
        rfc.set_params(n_estimators=s)
        rfc.fit(x_train, y_train)

        tr_score, te_score = score(rfc, train=(x_train, y_train), test=(x_test, y_test), pr=False)

        train_scores.append(tr_score)
        test_scores.append(te_score)

    return rfc, train_scores, test_scores


def plot_auc(steps, train_scores, test_scores):
    """Plot the train and test auc scores vs total number of model estimators"""

    plt.figure(figsize=(4, 4))
    plt.plot(steps, train_scores)
    plt.plot(steps, test_scores)
    plt.xlabel('n_estimators')
    plt.ylabel('auc')
    plt.legend(['train', 'test'])

In [ ]:
steps = np.arange(1, MAX_ESTIMATORS, 4)

In [ ]:
rfc = RandomForestClassifier(warm_start=True)

%time rfc, train_scores, test_scores = multiple_fit(x_pd.values, y_pd.values.squeeze(), steps=steps)

print(f"With {len(rfc.estimators_)}: {train_scores[-1]} | {test_scores[-1]}")
plot_auc(steps, train_scores, test_scores)

## Single incremental forest specs

In [ ]:
x_train, x_test, y_train, y_test = dask_tts(x, y, test_size=0.25)

x_train.chunks

#### Incremental forest
1 estimator per subset, 10 % per chunk, 1 pass through data.


In [ ]:
srfc = Incremental(StreamingRFC(n_estimators_per_chunk=1, max_n_estimators=np.inf))

srfc.fit(x_train, y_train, classes=[0, 1])

tr_score, te_score = score(srfc, train=(x_train, y_train), test=(x_test, y_test), pr=True)

#### Incremental forest
20 estimators per subset (different features), 10 % per chunk, 1 pass through data.

In [ ]:
srfc = Incremental(StreamingRFC(n_estimators_per_chunk=20, max_n_estimators=np.inf))

srfc.fit(x_train, y_train, classes=[0, 1])

tr_score, te_score = score(srfc, train=(x_train, y_train), test=(x_test, y_test), pr=True)

#### Forest of partial decision trees
1 estimator per subset with all features, 10 % per chunk, 1 pass through data.

In [ ]:
srfc = Incremental(StreamingRFC(
    n_estimators_per_chunk=1,
    max_n_estimators=np.max(steps),
    max_features=x.shape[1])
)

srfc.fit(x_train, y_train,
         classes=[0, 1])

tr_score, te_score = score(srfc, train=(x_train, y_train), test=(x_test, y_test), pr=True)

#### Forest of partial decision trees
20 estimator per subset with all features, 10 % per chunk, 1 pass through data.

Extra estimators shouldn't help here?

In [ ]:
srfc = Incremental(StreamingRFC(
    n_estimators_per_chunk=20,
    max_n_estimators=np.max(steps),
    max_features=x.shape[1])
)

srfc.fit(x_train, y_train, classes=[0, 1])

tr_score, te_score = score(srfc, train=(x_train, y_train), test=(x_test, y_test), pr=True)

### n estimators per chunk vs performance

Effect of increasing estimators per subset (with different set ups)

Function here add Incremental to supplied model, and uses .fit to refit the full model in each iteration.

The other functions (above and in PerformanceComparisons.ipynb) do incremental fits using warm start (either directly or via .partial_fit). 

This means the timing information cannot be directly compared!

In [ ]:
def multiple_dask_fit(x: np.ndarray, y: np.ndarray, steps=np.arange(1, 101, 2),
                      **kwargs) -> None:
    """
    Fit increasing number of estimators using .partial_fit on a subsample of the training data.
    Uses Dask by adding Incremental to model and calling fit. This refits the whole model one each
    iteration, so will be slower than the other test functions. Timing this function can only be compared
    to other calls of this function.
    
    The data passed to the Random forest fit by partial_fit is handled by dask and is sequential batches
    of data, rather than random samples (as used by inc_partial_fit in PerformanceComparisons.ipynb).
    
    StreamingRFC.n_estimators: Number of estimators that will be fit in each step. Set from first
                               difference in range (ie. range[1]-range[0])
    StreamingRFC.max_n_estimators: Limit on number of estimators than will be fit in model. Should >
                                   range[-1].
    
    :param steps: Range to iterate over. Sets total number of estimators that will be fit in model
                  after each iteration. Should be range with constant step size.
    """

    x_train, x_test, y_train, y_test = dask_tts(x, y, test_size=0.25)

    train_scores = []
    test_scores = []
    for s in steps:
        # Create fresh model each iteration
        srfc_ = StreamingRFC(n_estimators_per_chunk=s, max_n_estimators=np.inf, **kwargs)

        # Add Incremental
        srfc_ = Incremental(srfc_)

        # Fit model with these n ests
        # From scratch each time
        srfc_.fit(x_train, y_train,
                  classes=[0, 1])

        tr_score, te_score = score(
            srfc_,
            train=(x_train, y_train),
            test=(x_test, y_test),
            pr=False)

        train_scores.append(tr_score)
        test_scores.append(te_score)

    return srfc_, train_scores, test_scores

#### Incremental forest
*range* estimators per subset (different features), 10 % per chunk, 1 pass through data.

In [ ]:
steps = np.arange(1, MAX_ESTIMATORS, 6)

%time final_est, train_scores, test_scores = multiple_dask_fit(x, y, steps=steps)
print(f"With {len(final_est.estimators_)}: {train_scores[-1]} | {test_scores[-1]}")
plot_auc(steps, train_scores, test_scores)

#### Forest of partial decision trees
*range* estimators per subset with all features, 10 % per chunk, 1 pass through data.

In [ ]:
steps = np.arange(1, MAX_ESTIMATORS, 6)

%time final_est, train_scores, test_scores = multiple_dask_fit(x, y, steps=steps, max_features=x.shape[1])
print(f"With {len(final_est.estimators_)}: {train_scores[-1]} | {test_scores[-1]}")
plot_auc(steps, train_scores, test_scores)